In [ ]:
import os
import zipfile
import pandas as pd
from io import TextIOWrapper


def parse_zip(zip_path):
    dfs = {}
    with zipfile.ZipFile(zip_path) as z:
        for name in z.namelist():
            if not name.endswith(".dat"):
                continue

            base = os.path.basename(name)
            with z.open(name) as f:
                if base == "users.dat":
                    cols = ["UserID", "Gender", "Age", "Occupation", "ZipCode"]

                elif base == "ratings.dat":
                    cols = ["UserID", "MovieID", "Rating", "Timestamp"]

                elif base == "movies.dat":
                    cols = ["MovieID", "Title", "Genres"]

                df = pd.read_csv(
                    TextIOWrapper(f, encoding="latin-1"),
                    sep="::",
                    engine="python",
                    header=None,
                    names=cols,
                )

                dfs[base.replace(".dat", "")] = df

    return dfs


dfs = parse_zip("ml-1m.zip")
users = dfs["users"]
movies = dfs["movies"]
ratings = dfs["ratings"]

merged_df = ratings.merge(users, on="UserID").merge(movies, on="MovieID")
merged_df

merged_df

1.Требуется вычислить средние рейтинги для фильма с ID = № варианта в зависимости от пола;

Вариант 17

In [ ]:
variant = 17

my_film_df = merged_df[merged_df["MovieID"] == variant]
my_film_df.groupby("Gender")["Rating"].mean()

2.Отобрать фильмы, получившие не менее 250 оценок, и для них определить средний
рейтинг в зависимости от пола;
Результат вывести в виде DataFrame с колонками
['MovieID', 'Title', 'Rating_F', 'Rating_M']

In [ ]:
# Group movies by qnty of rates >=250
mov_id_gb = merged_df.groupby("MovieID").size()
mov_id_gb = mov_id_gb[mov_id_gb >= 250]

# Trim origin df by top movs
top_mov_df = merged_df[merged_df["MovieID"].isin(mov_id_gb.index)]


mean_rating_by_gender_gb = top_mov_df.groupby(["MovieID", "Title", "Gender"])[
    "Rating"
].mean()
mean_rating_by_gender_gb

# Assemble new df from mean ratings
mean_rating_df = mean_rating_by_gender_gb.unstack().reset_index().rename(columns={"F": "Rating_F", "M": "Rating_M"})
mean_rating_df.columns.name = None

mean_rating_df

3.Найти фильмы, оказавшиеся в топ 5 у зрителей-женщин и худшие 5 у зрителей-
мужчин. Найти фильмы (топ 5), по которым мужчины и женщины сильнее всего
разошлись в оценках;

In [23]:
top5_women = mean_rating_df.sort_values("Rating_F", ascending=False).head(5)
top5_women

top5_men = mean_rating_df.sort_values("Rating_M", ascending=True).head(5)
top5_men

mean_rating_df["Diff"] = (mean_rating_df["Rating_F"] - mean_rating_df["Rating_M"]).abs()
top_diff = mean_rating_df.sort_values("Diff", ascending=False).head(5)
top_diff

,MovieID,Title,Rating_F,Rating_M,Diff
313,1088,Dirty Dancing (1987),3.790378,2.959596,0.830782
352,1201,"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
1186,3760,"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
796,2468,Jumpin' Jack Flash (1986),3.254717,2.578358,0.676359
68,231,Dumb & Dumber (1994),2.697987,3.336595,0.638608


4.Определить фильмы (топ 5), вызвавшие в среднем наибольшее разногласие у зрителей
независимо от пола;

In [ ]:
movie_std = merged_df.groupby(["MovieID", "Title"])["Rating"].std()
movie_std = movie_std.reset_index().rename(columns={"Rating": "Rating_Deviation"})
movie_std.sort_values("Rating_Deviation", ascending=False).head(5)


5.Определить средний рейтинг каждого жанра для различных возрастов. Установите,
фильмы какого жанра в каждой категории возрастов имеют наибольший и наименьший
рейтинги.

In [ ]:
df_exploded = merged_df.copy()
df_exploded["Genres"] = df_exploded["Genres"].str.split("|").explode("Genres")

genre_age_mean = df_exploded.groupby(["Age", "Genres"])["Rating"].mean().reset_index()

max_by_age = genre_age_mean.iloc[genre_age_mean.groupby("Age")["Rating"].idxmax()]

min_by_age = genre_age_mean.iloc[genre_age_mean.groupby("Age")["Rating"].idxmin()]

,Age,Genres,Rating
0,1,Action,3.543712
1,1,Adventure,3.542649
2,1,Animation,3.578526
3,1,Children's,3.561672
4,1,Comedy,3.524950
...,...,...,...
121,56,Romance,3.773036
122,56,Sci-Fi,3.788780
123,56,Thriller,3.755461
124,56,War,3.734255
